# 📘 KoBERT 텍스트 분류 실습 (CSV 기반)

- 모델: `skt/kobert-base-v1`
- 데이터: `review.csv`
- 작업: 리뷰 텍스트 → 라벨 분류
- 학습/평가 → 정확도 측정

In [ ]:
# 0. 필요 라이브러리 설치
!pip install datasets transformers scikit-learn torch sentencepiece

In [ ]:
# 1. 라이브러리 임포트
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments
)
import torch
from sklearn.metrics import accuracy_score
import os

In [ ]:
# 2. KoBERT 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("skt/kobert-base-v1")
print("✅ KoBERT 토크나이저 로드 완료")

In [ ]:
# 3. CSV 파일 → Hugging Face Dataset 로드
dataset = load_dataset("csv", data_files={"train": "review.csv"})
print("✅ CSV 로드 완료")
print("📊 데이터 컬럼:", dataset["train"].column_names)

In [ ]:
# 4. 토크나이저 적용 함수
def tokenize_function(examples):
    return tokenizer(
        examples["review"],
        padding="max_length",
        truncation=True,
        max_length=128,
        return_token_type_ids=False
    )

In [ ]:
# 5. 전체 데이터셋에 토큰화 적용
tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(["review"])
tokenized_dataset.set_format("torch")
print("✅ 토크나이징 및 텐서 변환 완료")

In [ ]:
# 6. Train-Test Split (80% 훈련, 20% 테스트)
split_datasets = tokenized_dataset["train"].train_test_split(test_size=0.2)
tokenized_dataset = split_datasets
print("✅ Train/Test 분할 완료")
print("📊 Train 데이터 개수:", len(tokenized_dataset["train"]))
print("📊 Test 데이터 개수:", len(tokenized_dataset["test"]))

In [ ]:
# 7. KoBERT 모델 로드
model = AutoModelForSequenceClassification.from_pretrained(
    "skt/kobert-base-v1",
    num_labels=7  # 라벨 개수 (필요시 수정)
)
print("✅ KoBERT 모델 로드 완료")

def cast_to_int(example):
    example["label"] = int(example["label"])
    return example
tokenized_dataset = tokenized_dataset.map(cast_to_int)

In [ ]:
# CUDA 에러 추적용
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
# 8. Trainer 설정
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=50,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    save_total_limit=2,
    eval_strategy="epoch",
    logging_strategy="steps",
    logging_steps=10,
    save_strategy="epoch",
    logging_dir='./logs',
    save_steps=500,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"]
)
print("✅ Trainer 설정 완료")

In [ ]:
# 9. 모델 학습
print("🚀 학습 시작!")
trainer.train()
print("🚀 학습 완료!")

In [ ]:
# 10. 최종 평가
print("🔍 테스트 세트 평가 시작...")
predictions = trainer.predict(tokenized_dataset["test"])
preds = torch.argmax(torch.tensor(predictions.predictions), dim=1)
labels = tokenized_dataset["test"]["label"][:]

acc = accuracy_score(labels, preds.tolist())
print(f"🎯 최종 정확도: {acc:.4f}")